Importamos las librerias necesarias

In [1]:
import findspark
findspark.init('/opt/spark')

from datetime import datetime
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, date_format, udf 
from pyspark.sql.types import DateType
from pyspark.sql import Window
import pyspark.sql.functions as F
import numpy as np
from pyspark.ml.feature import MinMaxScaler
from pyspark.ml.feature import VectorAssembler
from pyspark.ml import Pipeline
from pyspark.sql.functions import udf
from pyspark.sql.types import DoubleType
from pyspark.ml.feature import OneHotEncoder, StringIndexer, VectorAssembler
import time


#path del archivo postgresql-42.1.4.jar
dir = "/Users/jasonsolano/Documents/BigData/BigDataTEC/Clase1/DB/"
spark = SparkSession \
    .builder \
    .appName("Basic JDBC pipeline") \
    .config("spark.driver.extraClassPath", dir+"postgresql-42.1.4.jar") \
    .config("spark.executor.extraClassPath", dir+"postgresql-42.1.4.jar") \
    .getOrCreate()

start = time.time()

Importamos los datos limpios de postgres

In [2]:
df = spark \
    .read \
    .format("jdbc") \
    .option("url", "jdbc:postgresql://localhost/ImmoDataBase") \
    .option("user", "postgres") \
    .option("password", "102800") \
    .option("dbtable", "apartmentrentalclean") \
    .load()

df.limit(5).toPandas().head()

,regio1,serviceCharge,heatingType,telekomTvOffer,newlyConst,balcony,picturecount,pricetrend,telekomUploadSpeed,totalRent,...,typeOfFlat,geo_plz,noRooms,floor,noRoomsRange,garden,livingSpaceRange,regio2,regio3,date
0,Nordrhein_Westfalen,100.00,central_heating,ONE_YEAR_FREE,False,False,29,3.52,2.4,429.00,...,ground_floor,45329,2.0,0.0,2,False,2,Essen,Karnap,May19
1,Nordrhein_Westfalen,84.00,central_heating,ONE_YEAR_FREE,False,True,4,4.92,2.4,578.00,...,roof_storey,48282,3.0,2.0,3,False,3,Steinfurt_Kreis,Emsdetten,Sep18
2,Nordrhein_Westfalen,115.00,night_storage_heater,ONE_YEAR_FREE,False,False,11,4.69,40.0,365.00,...,roof_storey,46242,3.0,3.0,3,False,2,Bottrop,Lehmkuhle,Sep18
3,Sachsen_Anhalt,33.00,self_contained_central_heating,ONE_YEAR_FREE,False,False,9,0.93,40.0,358.00,...,roof_storey,39218,2.0,0.0,2,False,2,Salzlandkreis,Schönebeck_Elbe,May19
4,Sachsen,169.75,gas_heating,ONE_YEAR_FREE,False,True,9,-1.89,2.4,449.49,...,apartment,9126,2.0,2.0,2,False,2,Chemnitz,Bernsdorf,May19


Se imprime el esquema

In [3]:
df.printSchema()

root
 |-- regio1: string (nullable = true)
 |-- serviceCharge: double (nullable = true)
 |-- heatingType: string (nullable = true)
 |-- telekomTvOffer: string (nullable = true)
 |-- newlyConst: boolean (nullable = true)
 |-- balcony: boolean (nullable = true)
 |-- picturecount: integer (nullable = true)
 |-- pricetrend: double (nullable = true)
 |-- telekomUploadSpeed: double (nullable = true)
 |-- totalRent: double (nullable = true)
 |-- firingTypes: string (nullable = true)
 |-- hasKitchen: boolean (nullable = true)
 |-- geo_bln: string (nullable = true)
 |-- cellar: boolean (nullable = true)
 |-- yearConstructedRange: double (nullable = true)
 |-- baseRent: double (nullable = true)
 |-- livingSpace: double (nullable = true)
 |-- geo_krs: string (nullable = true)
 |-- condition: string (nullable = true)
 |-- lift: boolean (nullable = true)
 |-- baseRentRange: integer (nullable = true)
 |-- typeOfFlat: string (nullable = true)
 |-- geo_plz: integer (nullable = true)
 |-- noRooms: doub

Creamos la siguiente función para normalizar las columnas numericas

In [4]:
def normalizeColumn(df,listColumnName):
    unlist = udf(lambda x: round(float(list(x)[0]),5), DoubleType())

    for i in listColumnName:
       
        assembler = VectorAssembler(inputCols=[i],outputCol=i+"_Vect")

        scaler = MinMaxScaler(inputCol=i+"_Vect", outputCol=i+"_Scaled")

        pipeline = Pipeline(stages=[assembler, scaler])
        df = pipeline.fit(df).transform(df).withColumn(i, unlist(i+"_Scaled")).drop(i+"_Vect")
        df = df.drop(i+"_Scaled")
    return df


Probamos la funcion para la normalización con solo una columna

Primero vemos la columna serviceCharge sin normalización

In [5]:
datasetSize = df.count()

df.groupBy('serviceCharge').count().sort(col("count").desc()).withColumn('perc_of_count_total', (F.col('count') / datasetSize)) \
  .toPandas().head(20)

,serviceCharge,count,perc_of_count_total
0,150.00,9009,0.053431
1,100.00,8066,0.047838
2,200.00,6811,0.040395
3,120.00,6615,0.039233
4,130.00,4809,0.028522
5,140.00,4587,0.027205
6,180.00,4202,0.024922
7,80.00,4148,0.024601
8,110.00,3927,0.023291
9,250.00,3914,0.023213


## Aplicamos la normalización con la función presentada anteriormente

In [6]:
numericColumns = list()
numericColumns.append('serviceCharge')
df = normalizeColumn(df,numericColumns)

Volvemos a consultar la columna para ver la normalización

In [7]:
df.groupBy('serviceCharge').count().sort(col("count").desc()).withColumn('perc_of_count_total', (F.col('count') / datasetSize)) \
  .toPandas().head(20)

,serviceCharge,count,perc_of_count_total
0,0.00103,9309,0.055211
1,0.00068,8417,0.049920
2,0.00137,6888,0.040852
3,0.00082,6727,0.039897
4,0.00096,4952,0.029370
5,0.00089,4906,0.029097
6,0.00055,4499,0.026683
7,0.00123,4409,0.026149
8,0.00075,4301,0.025509
9,0.00062,4220,0.025028


Una vez que se valida la normalización para dichos datos, correspondemos a realizarlo en las demas columnas numericas

# Normalización de columnas numéricas

In [8]:
# Columnas

columnas =  ['picturecount','pricetrend','telekomUploadSpeed','totalRent','yearConstructedRange'\
           ,'baseRent','livingSpace','baseRentRange','geo_plz','noRooms','floor','noRoomsRange',
           'livingSpaceRange']

print("Numero de columnas: ",len(columnas))

Numero de columnas:  13


## Aplicamos normalización de columnas

In [9]:
df = normalizeColumn(df,columnas)

In [10]:
df.select(columnas).toPandas().head()

,picturecount,pricetrend,telekomUploadSpeed,totalRent,yearConstructedRange,baseRent,livingSpace,baseRentRange,geo_plz,noRooms,floor,noRoomsRange,livingSpaceRange
0,0.26606,0.52677,0.01414,0.00003,0.000,0.00027,0.00047,0.125,0.44860,0.00179,0.001,0.25,0.16667
1,0.03670,0.58489,0.01414,0.00004,0.500,0.00037,0.00068,0.250,0.47839,0.00358,0.003,0.50,0.33333
2,0.10092,0.57534,0.39394,0.00002,0.125,0.00020,0.00040,0.000,0.45781,0.00358,0.004,0.50,0.16667
3,0.08257,0.41926,0.39394,0.00002,0.125,0.00017,0.00041,0.000,0.38696,0.00179,0.001,0.25,0.16667
4,0.08257,0.30220,0.01414,0.00003,0.000,0.00023,0.00054,0.000,0.08345,0.00179,0.003,0.25,0.16667


Como se observa, los datos numéricos se encuentran normalizados

# Iniciamos con la transformación de columnas booleansas

In [11]:
booleanColumns = ['newlyConst','balcony','hasKitchen','cellar','lift','garden']

print("Cantidad de columnas Booleanas: ",len(booleanColumns))

Cantidad de columnas Booleanas:  6


Función para remplazar columnas booleanas

In [12]:
def replaceBooleans(df,listColumns):
    for i in listColumns:
        df = df.withColumn(i,F.when((col(i) == True),1).otherwise(0))
    return df

In [13]:
df = replaceBooleans(df,booleanColumns)
df = df.drop()

Observamos la transformación`

In [14]:
df.select(booleanColumns).toPandas().head()

,newlyConst,balcony,hasKitchen,cellar,lift,garden
0,0,0,0,0,0,0
1,0,1,0,0,0,0
2,0,0,0,0,0,0
3,0,0,0,1,0,0
4,0,1,1,1,0,0


# Remplazamos variables categóricas por OneHotEncoding

Creamos la función para remplazar valores categóricos

In [15]:
def createCategoricalVects(df,columns):
    for i in columns:
        temp_indexer = StringIndexer(inputCol=i, outputCol=i+"_index").fit(df)
        df = temp_indexer.transform(df)
        temp_encoder = OneHotEncoder(inputCol=i+"_index", outputCol=i+"_vec")
        df = temp_encoder.transform(df)
        df = df.drop(i+"_index")
        print("Columna Procesada:",i,"Tammano",df.groupBy(i).count().count())
        df = df.drop(i)
    return df

Dicha función detecta si una columna tiene bastantes categorias para borrarla

Al observar la gran cantidad de columnas que se generan con diferentes datos categóricos, se evalua solo aceptar columnas categóricas con no más de 50 datos categoricos diferentes

In [16]:
def deleteBigColumns(df,columns,limit):
    goodColumns = list()
    for i in columns:
        size = df.groupBy(i).count().count()
        print(i,size)
        if(size > limit):
            print("Columna borrada:",i,"Tamanno:", size) 
            df = df.drop(i)
        else:
            goodColumns.append(i)
    return df,goodColumns

## Procesamos las columnas categóricas

Un detalle importante a rescatar es la escogencia del tamaño de las variables categóricas este tamaño se refiere a la cantidad de categorias que presenta varias columnas, por motivos de hardware y optmización se limite a un maximo de 50 categorias. Por otra parte este proceso fue detectado gracias a la validaciones que se realizaron en este jupyternotebook

In [17]:
categoryList = ['regio1','heatingType','geo_bln','geo_krs','condition','typeOfFlat','regio2','regio3','date','firingTypes','telekomTvOffer']
df,categoryList = deleteBigColumns(df,categoryList,50)
print('Procesamiento de las columnas: ')
df = createCategoricalVects(df,categoryList)


regio1 16
heatingType 13
geo_bln 16
geo_krs 419
Columna borrada: geo_krs Tamanno: 419
condition 10
typeOfFlat 10
regio2 419
Columna borrada: regio2 Tamanno: 419
regio3 7794
Columna borrada: regio3 Tamanno: 7794
date 3
firingTypes 102
Columna borrada: firingTypes Tamanno: 102
telekomTvOffer 3
Procesamiento de las columnas: 
Columna Procesada: regio1 Tammano 16
Columna Procesada: heatingType Tammano 13
Columna Procesada: geo_bln Tammano 16
Columna Procesada: condition Tammano 10
Columna Procesada: typeOfFlat Tammano 10
Columna Procesada: date Tammano 3
Columna Procesada: telekomTvOffer Tammano 3


## Observamos los nuevos datos categóricos

In [18]:
vectCat = [i+"_vec" for i in categoryList]
df.select(vectCat).show()

+---------------+---------------+---------------+-------------+--------------+-------------+------------------+
|     regio1_vec|heatingType_vec|    geo_bln_vec|condition_vec|typeOfFlat_vec|     date_vec|telekomTvOffer_vec|
+---------------+---------------+---------------+-------------+--------------+-------------+------------------+
| (15,[1],[1.0])| (12,[0],[1.0])| (15,[1],[1.0])|(9,[6],[1.0])| (9,[2],[1.0])|(2,[0],[1.0])|     (2,[0],[1.0])|
| (15,[1],[1.0])| (12,[0],[1.0])| (15,[1],[1.0])|(9,[0],[1.0])| (9,[1],[1.0])|    (2,[],[])|     (2,[0],[1.0])|
| (15,[1],[1.0])| (12,[8],[1.0])| (15,[1],[1.0])|(9,[0],[1.0])| (9,[1],[1.0])|    (2,[],[])|     (2,[0],[1.0])|
| (15,[2],[1.0])| (12,[4],[1.0])| (15,[2],[1.0])|(9,[0],[1.0])| (9,[1],[1.0])|(2,[0],[1.0])|     (2,[0],[1.0])|
| (15,[0],[1.0])| (12,[2],[1.0])| (15,[0],[1.0])|(9,[2],[1.0])| (9,[0],[1.0])|(2,[0],[1.0])|     (2,[0],[1.0])|
| (15,[1],[1.0])| (12,[0],[1.0])| (15,[1],[1.0])|(9,[2],[1.0])| (9,[2],[1.0])|(2,[1],[1.0])|     (2,[0],

# Mostramos la columnas que utilizamos para el training

In [19]:
datasetColumns = df.columns
datasetColumns.remove('totalRent')
print(datasetColumns)



['serviceCharge', 'newlyConst', 'balcony', 'picturecount', 'pricetrend', 'telekomUploadSpeed', 'hasKitchen', 'cellar', 'yearConstructedRange', 'baseRent', 'livingSpace', 'lift', 'baseRentRange', 'geo_plz', 'noRooms', 'floor', 'noRoomsRange', 'garden', 'livingSpaceRange', 'regio1_vec', 'heatingType_vec', 'geo_bln_vec', 'condition_vec', 'typeOfFlat_vec', 'date_vec', 'telekomTvOffer_vec']


In [20]:
df.select(datasetColumns).show()

+-------------+----------+-------+------------+----------+------------------+----------+------+--------------------+--------+-----------+----+-------------+-------+-------+-----+------------+------+----------------+---------------+---------------+---------------+-------------+--------------+-------------+------------------+
|serviceCharge|newlyConst|balcony|picturecount|pricetrend|telekomUploadSpeed|hasKitchen|cellar|yearConstructedRange|baseRent|livingSpace|lift|baseRentRange|geo_plz|noRooms|floor|noRoomsRange|garden|livingSpaceRange|     regio1_vec|heatingType_vec|    geo_bln_vec|condition_vec|typeOfFlat_vec|     date_vec|telekomTvOffer_vec|
+-------------+----------+-------+------------+----------+------------------+----------+------+--------------------+--------+-----------+----+-------------+-------+-------+-----+------------+------+----------------+---------------+---------------+---------------+-------------+--------------+-------------+------------------+
|       6.8E-4|       

# Creamos la columna features

In [21]:
df_assembler = VectorAssembler(inputCols = datasetColumns,outputCol="features")
df = df_assembler.transform(df)
df = df.select(['features','totalRent'])


In [22]:
df = df.withColumnRenamed('totalRent','label')

In [23]:

df.select(['features','label']).show()


+--------------------+------+
|            features| label|
+--------------------+------+
|(83,[0,3,4,5,9,10...|3.0E-5|
|(83,[0,2,3,4,5,8,...|4.0E-5|
|(83,[0,3,4,5,8,9,...|2.0E-5|
|(83,[0,3,4,5,7,8,...|2.0E-5|
|(83,[0,2,3,4,5,6,...|3.0E-5|
|(83,[0,3,4,5,7,8,...|3.0E-5|
|(83,[0,2,3,4,5,7,...|3.0E-5|
|(83,[0,2,3,4,5,9,...|2.0E-5|
|(83,[0,3,4,5,7,8,...|3.0E-5|
|(83,[0,2,3,4,5,6,...|7.0E-5|
|(83,[0,3,4,5,8,9,...|3.0E-5|
|(83,[0,2,3,4,5,7,...|7.0E-5|
|(83,[0,2,3,4,5,7,...|3.0E-5|
|(83,[0,2,3,4,5,8,...|4.0E-5|
|(83,[0,3,4,5,8,9,...|3.0E-5|
|(83,[0,2,3,4,5,6,...|7.0E-5|
|(83,[0,3,4,5,7,8,...|3.0E-5|
|(83,[0,2,3,4,5,6,...|3.0E-5|
|(83,[0,2,3,4,5,6,...|8.0E-5|
|(83,[0,2,3,4,5,7,...|3.0E-5|
+--------------------+------+
only showing top 20 rows



In [24]:
df.select("features").show(20,False)

+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|features                                                                                                                                                                                                 |
+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|(83,[0,3,4,5,9,10,12,13,14,15,16,18,20,34,47,67,72,79,81],[6.8E-4,0.26606,0.52677,0.01414,2.7E-4,4.7E-4,0.125,0.4486,0.00179,0.001,0.25,0.16667,1.0,1.0,1.0,1.0,1.0,1.0,1.0])                            |
|(83,[0,2,3,4,5,8,9,10,12,13,14,15,16,18,20,34,47,61,71,81],[5.7E-4,1.0,0.0367,0.58489,0.01414,0.5,3.7E-4,6.8E-4,0.25,0.47839,0.00358,0.003,0.5,0.33333,1.0,1.0,1.0,1.0,1.0,1.0])       

# Dividimos el dataset para entrenamiento y prueba

In [25]:
train,test = df.randomSplit([0.8,0.2])

# Iniciamos con el entrenamiento

In [26]:
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator
from pyspark.ml.evaluation import BinaryClassificationEvaluator

## Iniciamos con el algoritmo Linear Regression

In [27]:
# Importacion de libs y operaciones
from pyspark.ml.regression import LinearRegression
lr = LinearRegression()

In [28]:
lr_model=lr.fit(train)

In [29]:
predictions_df = lr_model.transform(test)

In [30]:
# evaluacion del modelo, le llamaremos model_predictions
model_predictions = lr_model.evaluate(test)
model_predictions

### imprimimos el valor R2:

In [31]:
round(float(model_predictions.r2),5)

0.92916

### imprimimos el valor de meanSquaredError

In [32]:
round(float(model_predictions.meanSquaredError),5)

0.0

## Probamos el Algoritmo Decision Tree Regressor

In [33]:
from pyspark.ml.regression import DecisionTreeRegressor

In [34]:
dec_tree = DecisionTreeRegressor()

In [35]:
dec_tree_model = dec_tree.fit(train)

In [36]:
model_predictions = dec_tree_model.transform(test)

Usamos el RegressionEvaluator

In [37]:
from pyspark.ml.evaluation import RegressionEvaluator


In [38]:
# R2 value of the model on test data 
dt_evaluator = RegressionEvaluator(metricName='r2')
dt_r2 = dt_evaluator.evaluate(model_predictions)
print(f'The r-square value this model is {round(float(dt_r2),5)}')

# RMSE value of the model on test data 
dt_evaluatorRMSE = RegressionEvaluator(metricName='rmse')
dt_RMSE = dt_evaluatorRMSE.evaluate(model_predictions)
print(f'The RMSE value this model is {round(float(dt_RMSE),5)}')

# RMSE value of the model on test data 
dt_evaluatorMAE = RegressionEvaluator(metricName='mae')
dt_mae = dt_evaluatorMAE.evaluate(model_predictions)
print(f'The MAE value this model is {round(float(dt_RMSE),5)}')

The r-square value this model is -0.15439
The RMSE value this model is 0.00037
The MAE value this model is 0.00037


## Utilizamos el algoritmo Random Forest Regressor

In [39]:
from pyspark.ml.regression import RandomForestRegressor

# Regresor 
Rfr = RandomForestRegressor()
# Train model, le llamaremos rf_model
rf_model=Rfr.fit(train)
# model_predictions
model_predictions = rf_model.transform(test)

Observamos el reseltado

In [40]:
# R2 value of the model on test data 
dt_evaluator = RegressionEvaluator(metricName='r2')
dt_r2 = dt_evaluator.evaluate(model_predictions)
print(f'The r-square value this model is {round(float(dt_r2),5)}')

# RMSE value of the model on test data 
dt_evaluatorRMSE = RegressionEvaluator(metricName='rmse')
dt_RMSE = dt_evaluatorRMSE.evaluate(model_predictions)
print(f'The RMSE value this model is {round(float(dt_RMSE),5)}')

# RMSE value of the model on test data 
dt_evaluatorMAE = RegressionEvaluator(metricName='mae')
dt_mae = dt_evaluatorMAE.evaluate(model_predictions)
print(f'The MAE value this model is {round(float(dt_RMSE),5)}')

The r-square value this model is -1.24016
The RMSE value this model is 0.00052
The MAE value this model is 0.00052


## Utilizamos Gradient-Boosted Tree Regressor¶

In [41]:
from pyspark.ml.regression import GBTRegressor
gbt = GBTRegressor()
gbt_model=gbt.fit(train)
model_predictions = gbt_model.transform(test)


In [42]:
# R2 value of the model on test data 
dt_evaluator = RegressionEvaluator(metricName='r2')
dt_r2 = dt_evaluator.evaluate(model_predictions)
print(f'The r-square value this model is {round(float(dt_r2),5)}')

# RMSE value of the model on test data 
dt_evaluatorRMSE = RegressionEvaluator(metricName='rmse')
dt_RMSE = dt_evaluatorRMSE.evaluate(model_predictions)
print(f'The RMSE value this model is {round(float(dt_RMSE),5)}')

# RMSE value of the model on test data 
dt_evaluatorMAE = RegressionEvaluator(metricName='mae')
dt_mae = dt_evaluatorMAE.evaluate(model_predictions)
print(f'The MAE value this model is {round(float(dt_RMSE),5)}')

The r-square value this model is -0.01908
The RMSE value this model is 0.00035
The MAE value this model is 0.00035


## Utilizamos Isotonic regression


In [43]:
from pyspark.ml.regression import IsotonicRegression
ir_model = IsotonicRegression().fit(train)
model_predictions = ir_model.transform(test)

In [44]:
# R2 value of the model on test data 
dt_evaluator = RegressionEvaluator(metricName='r2')
dt_r2 = dt_evaluator.evaluate(model_predictions)
print(f'The r-square value this model is {round(float(dt_r2),5)}')

# RMSE value of the model on test data 
dt_evaluatorRMSE = RegressionEvaluator(metricName='rmse')
dt_RMSE = dt_evaluatorRMSE.evaluate(model_predictions)
print(f'The RMSE value this model is {round(float(dt_RMSE),5)}')

# RMSE value of the model on test data 
dt_evaluatorMAE = RegressionEvaluator(metricName='mae')
dt_mae = dt_evaluatorMAE.evaluate(model_predictions)
print(f'The MAE value this model is {round(float(dt_RMSE),5)}')

The r-square value this model is -0.00028
The RMSE value this model is 0.00035
The MAE value this model is 0.00035


In [45]:
end = time.time()
print("Tiempo Transcurrido en minutos: ",(end-start)/60)

Tiempo Transcurrido en minutos:  20.587393816312154


# Conclusión

Las pruebas se realizaron con un dataset 20% para validación y 80% para entrenamiento con un total de 168609 registros

Como se puede observar lo mas alarmante es el resultado de la metrica r_2, que tiende a cero o en algunos modelos hasta en negativo, estamos utilizando datos categoricos y bastantes columnas como un total de 83, la metrica r_2 puede verse muy afectada por la distancias que se encuentras los vectores categoricos, por lo cual presenta un valor inconsistente.

Este articulo nos menciona como esta metrica no nos puede contar si un modelo de regression esta bien entrenado:

https://blog.minitab.com/blog/adventures-in-statistics-2/regression-analysis-how-do-i-interpret-r-squared-and-assess-the-goodness-of-fit

MAE: el error se calcula como un promedio de diferencias absolutas entre los valores objetivo y las predicciones.

RMSE: básicamente mide el error cuadrado promedio de nuestras predicciones


Por otra parte La metrica MAE Y RMSE, son las metricas mas usadas para modelos de regresición, por lo cual entre mas bajo sea este valor mejor, podemos observar que en los modelos el MAE y RMSE son muy bajos teniendo como mejores modelos Isotonic regression y Gradient-Boosted Tree Regressor los cuales los valoraremos por separado en otra jupyternotebook 